# Report for data wrangling

7 february 2023\
Ali Benchemsi\
Gerard van Konijnenburg

# Libraries and options

In [2]:
import numpy as np
import pandas as pd
from sodapy import Socrata
from configparser import ConfigParser
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

# Data loading & wrangling

Idee Gerard zaterdag 21-1

We hebben een heleboel data van taxiritten per rit, maar weerdata per dag voor ruim 100 weerstations. Ik stel voor om taxiritten te aggregeren tot 1 record per dag, zone van vertrek en zone van aankomst. Dit kunnen we dan tegen weerdata aanhouden en kijken of er een relatie in zit.

De weerdata heeft veel gegevens over het weer, dus daar moeten we ook wat zinnigs over vinden. Ik ben al met de taxiritten aan het stoeien, wil jij je eerst richten op de weerdata?

## Yellow cab data

In [3]:
#Read app token from config file
parser = ConfigParser()
parser.read('configuration.txt')
app_token = parser.get('ny_opendata', 'app_token')

In [15]:
#Load taxirides data

#Te implementeren
#Via de SQL call
#Transform the pickup timestamp to pickup date 
#Group by pickup date, pickup location, destination location
#Select count(rides), avg(passenger), avg(distance), avg(passenger * distance) plus grouping variables

#Voorbeelden
#https://github.com/xmunoz/sodapy/tree/master/examples
#https://github.com/xmunoz/sodapy/blob/master/examples/soql_queries.ipynb

#Implementatie
client = Socrata("data.cityofnewyork.us", app_token)
dataset = "m6nq-qud6" #2021 Yellow Taxi Trip Data

result = client.get(dataset, limit = 100)
df_taxirides = pd.DataFrame(result)
#results = client.get(dataset, 
#    group = "pulocationid",
#    select = "sum(passenger_count), sum(passenger_count * trip_distance)",                
#    limit=100)

In [16]:
# Taxirides debug, to be removed later
df_taxirides.head()
#type(test[1])

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2021-01-01T00:30:10.000,2021-01-01T00:36:12.000,1,2.10,1,N,142,43,2,8,3,0.5,0,0,0.3,11.8,2.5
1,1,2021-01-01T00:51:20.000,2021-01-01T00:52:19.000,1,0.20,1,N,238,151,2,3,0.5,0.5,0,0,0.3,4.3,0
2,1,2021-01-01T00:43:30.000,2021-01-01T01:11:06.000,1,14.70,1,N,132,165,1,42,0.5,0.5,8.65,0,0.3,51.95,0
3,1,2021-01-01T00:15:48.000,2021-01-01T00:31:01.000,0,10.60,1,N,138,132,1,29,0.5,0.5,6.05,0,0.3,36.35,0
4,2,2021-01-01T00:31:49.000,2021-01-01T00:48:21.000,1,4.94,1,N,68,33,1,16.5,0.5,0.5,4.06,0,0.3,24.36,2.5


In [6]:
#Taxirides data wrangling

#Te implementeren
#colommen naar juiste datatype, alles staat in string

## Taxi zones

In [7]:
#Load taxizone data

#Te implementeren
#bron: https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc
#download data in een geschikt formaat voor data wrangling hieronder
#uitzoeken of er iets is met longitude, latitude data er al in
#data lezen

In [8]:
#Taxizone data wrangling

#Te implementeren
#bereken latitude and longitude voor elke zone zodat we in een volgende stap afstand tot de weerstations kunnen bepalen

## Weather data

In [4]:
#Load weather data

#Te implementeren
#lees weerdata (zie in overleaf map 'data')
#implementatie geeft nu een waarschuwing voor mixed datatupe in colom 29 en 55, uitzoeken hoe dit te ondervangen

#Implemenatie
df_weather = pd.read_csv('data\weatherdata.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (29,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#Weather data wrangling

#Te implementeren
#geen idee, nog niet naar gekeken